In [1]:
# Dependencies
import pandas as pd
import sqlite3
from pyspark import SparkContext
sc =SparkContext()
from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)

In [2]:
# Create connections to database
conn = sqlite3.connect("Data/Hotels.db")

#Load the database table into a pandas dataframe
ratings = pd.read_sql_query("select * from ratings;", conn)
conn.close()

# Preview the dataframe
ratings.head()

,index,name,reviews_date,reviews_rating,reviews_sourceURLs,reviews_text,reviews_title,reviews_userCity,reviews_userProvince
0,0,Rancho Valencia Resort Spa,2013-11-14T00:00:00Z,5.0,www.hotels.com,Our experience at Rancho Valencia was absolute...,Best romantic vacation ever!!!!,None,None
1,1,Rancho Valencia Resort Spa,2014-07-06T00:00:00Z,5.0,www.hotels.com,Amazing place. Everyone was extremely warm and...,Sweet sweet serenity,None,None
2,2,Rancho Valencia Resort Spa,2015-01-02T00:00:00Z,5.0,www.hotels.com,We booked a 3 night stay at Rancho Valencia to...,Amazing Property and Experience,None,None
3,3,Aloft Arundel Mills,2016-05-15T00:00:00Z,2.0,www.tripadvisor.com,Currently in bed writing this for the past hr ...,"Never again...beware, if you want sleep.",Richmond,VA
4,4,Aloft Arundel Mills,2016-07-09T00:00:00Z,5.0,www.tripadvisor.com,I live in Md and the Aloft is my Home away fro...,ALWAYS GREAT STAY...,Laurel,MD


In [3]:
# Remove columns that will not be used in the analysis
df = ratings.drop(columns=["reviews_date","reviews_sourceURLs","reviews_title","reviews_userCity","reviews_userProvince"],axis=1)

# Preview the dataframe
df.head()

,index,name,reviews_rating,reviews_text
0,0,Rancho Valencia Resort Spa,5.0,Our experience at Rancho Valencia was absolute...
1,1,Rancho Valencia Resort Spa,5.0,Amazing place. Everyone was extremely warm and...
2,2,Rancho Valencia Resort Spa,5.0,We booked a 3 night stay at Rancho Valencia to...
3,3,Aloft Arundel Mills,2.0,Currently in bed writing this for the past hr ...
4,4,Aloft Arundel Mills,5.0,I live in Md and the Aloft is my Home away fro...


In [4]:
spark_ratings = sqlContext.createDataFrame(df)
spark_ratings.show(10)

+-----+--------------------+--------------+--------------------+
|index|                name|reviews_rating|        reviews_text|
+-----+--------------------+--------------+--------------------+
|    0|Rancho Valencia R...|           5.0|Our experience at...|
|    1|Rancho Valencia R...|           5.0|Amazing place. Ev...|
|    2|Rancho Valencia R...|           5.0|We booked a 3 nig...|
|    3| Aloft Arundel Mills|           2.0|Currently in bed ...|
|    4| Aloft Arundel Mills|           5.0|I live in Md and ...|
|    5| Aloft Arundel Mills|           5.0|I stayed here wit...|
|    6| Aloft Arundel Mills|           5.0|Beautiful rooms a...|
|    7| Aloft Arundel Mills|           5.0|We stayed here wh...|
|    8| Aloft Arundel Mills|           5.0|I travel a lot wi...|
|    9|Hampton Inn Suite...|           5.0|In my line of wor...|
+-----+--------------------+--------------+--------------------+
only showing top 10 rows



In [5]:
from pyspark.sql.functions import length
data = spark_ratings.withColumn('length', length(spark_ratings['reviews_text']))
data.show()

+-----+--------------------+--------------+--------------------+------+
|index|                name|reviews_rating|        reviews_text|length|
+-----+--------------------+--------------+--------------------+------+
|    0|Rancho Valencia R...|           5.0|Our experience at...|   169|
|    1|Rancho Valencia R...|           5.0|Amazing place. Ev...|   283|
|    2|Rancho Valencia R...|           5.0|We booked a 3 nig...|   471|
|    3| Aloft Arundel Mills|           2.0|Currently in bed ...|   198|
|    4| Aloft Arundel Mills|           5.0|I live in Md and ...|   307|
|    5| Aloft Arundel Mills|           5.0|I stayed here wit...|   244|
|    6| Aloft Arundel Mills|           5.0|Beautiful rooms a...|   340|
|    7| Aloft Arundel Mills|           5.0|We stayed here wh...|   213|
|    8| Aloft Arundel Mills|           5.0|I travel a lot wi...|   311|
|    9|Hampton Inn Suite...|           5.0|In my line of wor...|   307|
|   10|Hampton Inn Suite...|           5.0|The staff is very...|

In [6]:
from pyspark.ml.feature import Tokenizer, StopWordsRemover, HashingTF, IDF, StringIndexer

In [7]:
tokenizer = Tokenizer(inputCol="reviews_text", outputCol="token_text")
stopremove = StopWordsRemover(inputCol='token_text',outputCol='stop_tokens')
hashingTF = HashingTF(inputCol="token_text", outputCol='hash_token')
idf = IDF(inputCol='hash_token', outputCol='idf_token')

In [8]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.linalg import Vector

In [9]:
# Create feature vectors
clean_up = VectorAssembler(inputCols=['idf_token', 'length'], outputCol='features')

In [10]:
 # Create a and run a data processing Pipeline
from pyspark.ml import Pipeline
data_prep_pipeline = Pipeline(stages=[tokenizer, stopremove, hashingTF, idf, clean_up])

In [11]:
 # Fit and transform the pipeline
cleaner = data_prep_pipeline.fit(data)
cleaned = cleaner.transform(data)

Py4JJavaError: An error occurred while calling o50.fit.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 5 in stage 2.0 failed 1 times, most recent failure: Lost task 5.0 in stage 2.0 (TID 7, localhost, executor driver): org.apache.spark.SparkException: Failed to execute user defined function($anonfun$createTransformFunc$1: (string) => array<string>)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage1.processNext(Unknown Source)
	at org.apache.spark.sql.execution.BufferedRowIterator.hasNext(BufferedRowIterator.java:43)
	at org.apache.spark.sql.execution.WholeStageCodegenExec$$anonfun$11$$anon$1.hasNext(WholeStageCodegenExec.scala:619)
	at scala.collection.Iterator$$anon$11.hasNext(Iterator.scala:409)
	at scala.collection.Iterator$$anon$11.hasNext(Iterator.scala:409)
	at scala.collection.Iterator$$anon$11.hasNext(Iterator.scala:409)
	at scala.collection.Iterator$class.foreach(Iterator.scala:891)
	at scala.collection.Abs